In [8]:
import sys
# sys.path.insert(0, '..')
import shutil
import numpy as np
from sklearn.metrics import roc_auc_score
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.models import vgg19
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchsummary import summary
from utils import *
import warnings 
import os
warnings.filterwarnings("ignore")
import cv2
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
base_path = os.path.dirname(os.path.abspath('__file__'))
data_path = os.path.join(base_path, 'data','data_for_vgg_augmented')
train_path = os.path.join(data_path,'train')
test_path = os.path.join(data_path,'test')

In [5]:
with open('classes.txt', 'rb') as f:
    classes = pickle.load(f)

In [10]:
# model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, autoshape=False, force_reload=True, classes=len(classes))

In [30]:
img_width, img_height = 224, 224
device = "cuda" if torch.cuda.is_available() else "cpu"

model_yolov5 = torch.hub.load(repo_or_dir = f'C:\\Users\\yaron\\projects\\BGU-lior_rokach\\exe3\\yolov5',model = 'custom', path = 'best.pt',force_reload=True,source='local')

YOLOv5  v7.0-193-g485da42 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 117 layers, 4297350 parameters, 0 gradients, 10.5 GFLOPs
WARNING  YOLOv5 ClassificationModel is not yet AutoShape compatible. You must pass torch tensors in BCHW to this model, i.e. shape(1,3,224,224).


In [37]:
summary(model_yolov5, (3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 122, 122]           3,488
              SiLU-2         [-1, 32, 122, 122]               0
              Conv-3         [-1, 32, 122, 122]               0
            Conv2d-4           [-1, 64, 61, 61]          18,496
              SiLU-5           [-1, 64, 61, 61]               0
              Conv-6           [-1, 64, 61, 61]               0
            Conv2d-7           [-1, 32, 61, 61]           2,080
              SiLU-8           [-1, 32, 61, 61]               0
              Conv-9           [-1, 32, 61, 61]               0
           Conv2d-10           [-1, 32, 61, 61]           1,056
             SiLU-11           [-1, 32, 61, 61]               0
             Conv-12           [-1, 32, 61, 61]               0
           Conv2d-13           [-1, 32, 61, 61]           9,248
             SiLU-14           [-1, 32,

In [40]:
base_path = os.path.dirname(os.path.abspath('__file__'))
labels_file = os.path.join(base_path,'data')
data_path = os.path.join(base_path, 'data','data_for_vgg_augmented')
data_path

'C:\\Users\\yaron\\projects\\BGU-lior_rokach\\exe3\\data\\data_for_vgg_augmented'

In [41]:
def run_the_model(model_yolov5):
    print('running yolov5 model')    
    
    # Define the transformation for the input images
    transform = transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load the dataset
    train_data_dir = os.path.join(data_path, 'train')
    validation_data_dir = os.path.join(data_path, 'test')

    train_dataset = ImageFolder(train_data_dir, transform=transform)
    validation_dataset = ImageFolder(validation_data_dir, transform=transform)

    # Create data loaders
    batch_size = 4
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # Move the model to the device
    model_yolov5 = model_yolov5.to(device)

    
    # Define the loss function
    loss_fn = nn.CrossEntropyLoss()
    
    # log statistics
    train_accuracy_per_epoch = []
    train_cross_entropy_per_epoch = []
    test_accuracy_per_epoch = []
    test_cross_entropy_per_epoch = []

    train_predicted_labels = []
    train_true_labels = []
        

    model_yolov5.eval()
    with torch.no_grad():
        train_loss, train_acc = 0, 0
        for inputs, y_true in train_loader:
            inputs = inputs.to(device)
            y_true = y_true.to(device)

            y_pred = model_yolov5(inputs)

            loss = loss_fn(y_pred, y_true)
            train_loss += loss.item() 




            # Calculate and accumulate accuracy metric across all batches
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            train_acc += (y_pred_class == y_true).sum().item()/len(y_pred)

            train_predicted_labels.append(y_pred_class)
            train_true_labels.append(y_true)

         # Adjust metrics to get average loss and accuracy per batch 
        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)

        

    train_cross_entropy_per_epoch.append(train_loss)
    train_accuracy_per_epoch.append(train_acc)


    model_yolov5.eval()
    
    test_predicted_labels = []
    test_true_labels = []
    
    with torch.no_grad():
        test_loss, test_acc = 0, 0

        for inputs, y_true in validation_loader:
            inputs = inputs.to(device)
            y_true = y_true.to(device)

            y_pred = model_yolov5(inputs)

            loss = loss_fn(y_pred, y_true)
            test_loss += loss.item()


            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            test_acc += (y_pred_class == y_true).sum().item()/len(y_pred)

            test_predicted_labels.append(y_pred_class)
            test_true_labels.append(y_true)
            
        # Adjust metrics to get average loss and accuracy per batch 
        test_loss = test_loss / len(validation_loader)
        test_acc = test_acc / len(validation_loader)


        test_cross_entropy_per_epoch.append(test_loss)
        test_accuracy_per_epoch.append(test_acc)

    
    return train_cross_entropy_per_epoch, test_cross_entropy_per_epoch,  \
            train_predicted_labels, train_true_labels, test_predicted_labels, test_true_labels




In [42]:
train_cross_entropy_per_epoch, test_cross_entropy_per_epoch, \
train_predicted_labels, train_true_labels, test_predicted_labels, test_true_labels = run_the_model(model_yolov5)

In [57]:
train_predicted_labels_numpy = torch.concatenate(train_predicted_labels).detach().cpu().numpy()
train_true_labels_numpy = torch.concatenate(train_true_labels).detach().cpu().numpy()

test_predicted_labels_numpy = torch.concatenate(test_predicted_labels).detach().cpu().numpy()
test_true_labels_numpy = torch.concatenate(test_true_labels).detach().cpu().numpy()

In [58]:
from sklearn.metrics import confusion_matrix

In [68]:
confusion_matrix(train_true_labels_numpy, train_predicted_labels_numpy)

array([[ 72,   0,   1, ...,   0,   0,   0],
       [  0, 189,   0, ...,   0,   0,   0],
       [  1,   0,  70, ...,   1,   0,   0],
       ...,
       [  0,   1,   0, ..., 136,   0,   0],
       [  0,   0,   0, ...,   0,  33,   0],
       [  0,   0,   0, ...,   0,   0, 100]], dtype=int64)

In [69]:
confusion_matrix(test_true_labels_numpy, test_predicted_labels_numpy)

array([[13,  0,  0, ...,  0,  0,  0],
       [ 0, 22,  0, ...,  0,  0,  0],
       [ 1,  0, 15, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 23,  0,  0],
       [ 0,  0,  0, ...,  0,  4,  0],
       [ 0,  0,  0, ...,  0,  0, 13]], dtype=int64)

In [65]:
labels_names = []
train_path = os.path.join(data_path, 'train')
for class_name in os.listdir(train_path):
    labels_names.append(class_name)

In [70]:
# labels_names

In [84]:
from scikitplot.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

In [86]:
# plot_confusion_matrix(test_true_labels_numpy, test_predicted_labels_numpy)
# plt.show()
accuracy_score(test_true_labels_numpy, test_predicted_labels_numpy), accuracy_score(train_true_labels_numpy, train_predicted_labels_numpy)

(0.33276325433667236, 0.36087472098214285)

In [95]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)
# with numpy.printoptions(threshold=numpy.inf):
#     print(arr)

In [94]:
confusion_matrix(test_true_labels_numpy, test_predicted_labels_numpy).diagonal().sum(), confusion_matrix(train_true_labels_numpy, train_predicted_labels_numpy).diagonal().sum()

(1362, 10347)

In [96]:
save_path = os.path.join(base_path, 'yolo ypred ytrue')
np.savetxt(os.path.join(save_path,'train_true_labels_numpy.txt'), train_true_labels_numpy )
np.savetxt(os.path.join(save_path,'train_predicted_labels_numpy.txt'), train_predicted_labels_numpy)
np.savetxt(os.path.join(save_path,'test_true_labels_numpy.txt'), test_true_labels_numpy)
np.savetxt(os.path.join(save_path,'test_predicted_labels_numpy.txt'), test_predicted_labels_numpy)

In [11]:
save_path = os.path.join(base_path, 'yolo ypred ytrue')
train_true_labels_numpy = np.loadtxt(os.path.join(save_path,'train_true_labels_numpy.txt'))
train_predicted_labels_numpy = np.loadtxt(os.path.join(save_path,'train_predicted_labels_numpy.txt'))
test_true_labels_numpy = np.loadtxt(os.path.join(save_path,'test_true_labels_numpy.txt'))
test_predicted_labels_numpy = np.loadtxt(os.path.join(save_path,'test_predicted_labels_numpy.txt'))


In [12]:
train_true_labels_numpy

array([         30,          28,          39, ...,          58,          20,          66])

In [80]:
!python C:\\Users\\yaron\projects\\BGU-lior_rokach\\exe3\\yolov5\\train.py --img 640 --batch 16 --epochs 3 --data my_yolov5s.yaml --weights yolov5s.pt

C:\Users\yaron\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
Traceback (most recent call last):
  File "C:\Users\yaron\projects\BGU-lior_rokach\exe3\yolov5\train.py", line 61, in <module>
    from utils.loggers import Loggers
  File "C:\Users\yaron\projects\BGU-lior_rokach\exe3\yolov5\utils\loggers\__init__.py", line 15, in <module>
    from utils.loggers.wandb.wandb_utils import WandbLogger
  File "C:\Users\yaron\projects\BGU-lior_rokach\exe3\yolov5\utils\loggers\wandb\wandb_utils.py", line 23, in <module>
    import wandb
  File "C:\Users\yaron\anaconda3\lib\site-packages\wandb\__init__.py", line 197, in <module>
    from .analytics import Sentry as _Sentry
  File "C:\Users\yaron\anaconda3\lib\site-packages\wandb\analytics\__init__.py", line 3, in <module>
    from .sentry import Sentry
  File "C:\Users\yaron\ana

In [1]:
classes

NameError: name 'classes' is not defined